#Taking outputs from ABodyBuilder and MAPT, and embedding associated antibody sequences in Ankh and Ablang.

Ablang heavy and light chain embeddings will be concatenated along the row axis, and the combination of features will be used for a regression task to predict hydrophobicity and MAPT score.

In [1]:
#@title Mount drive and load libraries
%%capture
!pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install ankh

import os
import pandas as pd, numpy as np
from google.colab import drive

drive.mount('/content/drive/')
path = '/content/drive/MyDrive/msc-project-mbalmf01'
os.chdir(path)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230716_human_paired_seqs.csv', dtype={'Run':np.str_}, index_col=0)
print(df.columns)

In [ ]:
mapt = pd.read_csv('MAPT/230811_MAPT_scores_abs.csv')
print(mapt.describe())
print(mapt.columns)

         Acidics Fv     Basics Fv  Charge pH5 Fv  Charge pH5 H  Charge pH5 L  \
count  61029.000000  61029.000000   61029.000000  61029.000000  61028.000000   
mean      17.051058     18.206361       2.785151      1.985941      0.799223   
std        2.516573      2.124728       2.677337      2.233245      1.886530   
min        6.000000     10.000000     -12.000000    -11.000000    -10.000000   
25%       15.000000     17.000000       1.000000      1.000000      0.000000   
50%       17.000000     18.000000       3.000000      2.000000      1.000000   
75%       19.000000     20.000000       5.000000      3.000000      2.000000   
max       29.000000     28.000000      14.000000     11.000000      9.000000   

       Charge pH7 Fv  Charge pH7 H  Charge pH7 L  Hydrophobic Surface Fv  \
count   61029.000000  61029.000000  61028.000000            61029.000000   
mean        1.155303      1.103082      0.052222              581.804072   
std         2.521592      2.099052      1.806366   

In [ ]:
linker = 'SGGSTITSYNVYYTKLSSSGT'

mapt['scfv'] = mapt['Model Seq L'] + [linker]*mapt.shape[0] + mapt['Model Seq H']

df = df[df['scfv'].isin(mapt['scfv'].to_list())]

In [ ]:
df.to_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230811_human_paired_seqs_MAPT.csv', index=None)

In [2]:
df = pd.read_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230811_human_paired_seqs_MAPT.csv', dtype={'Run':np.str_})

In [ ]:
df.head()

,Unnamed: 1,sequence_id_heavy,ANARCI_status_heavy,sequence_heavy,sequence_alignment_aa_heavy,v_call_heavy,d_call_heavy,j_call_heavy,sequence_id_light,ANARCI_status_light,sequence_light,v_call_light,j_call_light,sequence_alignment_aa_light,Run,seq_id,scfv
0,2696,AGTAGTCTCCAAATGC-1_contig_1,"|Deletions: 10, 73||||",ACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCT...,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLE...,IGHV4-59*01,IGHD1-1*01,IGHJ4*02,AGTAGTCTCCAAATGC-1_contig_2,|||||,AGCTTCAGCTGTGGGTAGAGAAGACAGGACTCAGGACAATCTCCAG...,IGLV1-44*01,IGLJ1*01,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPK...,1287185,AGTAGTCTCCAAATGC-1_contig_1_AGTAGTCTCCAAATGC-1...,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPK...
1,2720,AGTCTTTCAGATCCAT-1_contig_1,"|Deletions: 10, 73||||",CGAGCCCAGCACTGGAAGTCGCCGGTGTTTCCATTCGGTGATCATC...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYAMHWVRQAPGKGLE...,IGHV3-30-3*01,IGHD3-16*02,IGHJ4*02,AGTCTTTCAGATCCAT-1_contig_2,|||||,GCTGGGGTCTCAGGAGGCAGCGCTCTCAGGACATCTCCACCATGGC...,IGLV2-14*01,IGLJ2*01,QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAP...,1287185,AGTCTTTCAGATCCAT-1_contig_1_AGTCTTTCAGATCCAT-1...,QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAP...
2,2729,AGTCTTTGTCCGCTGA-1_contig_2,"|Deletions: 10, 73||||",GGGGAGCTCTGGGAGAGGAGCCCCAGCCCTGAGATTCCCAGGTGTT...,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,IGHV3-9*01,IGHD1-26*01,IGHJ4*02,AGTCTTTGTCCGCTGA-1_contig_1,|||||,AGCTTCAGCTGTGGGTAGAGAAGACAGGACTCAGGACAATCTCCAG...,IGLV1-44*01,IGLJ2*01,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPK...,1287185,AGTCTTTGTCCGCTGA-1_contig_2_AGTCTTTGTCCGCTGA-1...,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPK...
3,2760,AGTGAGGGTTACGCGC-1_contig_1,"|Deletions: 10, 73||||",GGGGAGTGACTCCTGTGCCCCACCATGGACACACTTTGCTCCACGC...,QITLKESGPTLVKPTQTLTLTCTFSGFSLSTSGVGVGWIRQPPGKA...,IGHV2-5*02,IGHD2-15*01,IGHJ5*02,AGTGAGGGTTACGCGC-1_contig_2,|||||,GGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTG...,IGLV2-23*02,IGLJ2*01,QSALTQPASVSGSPGQSITISCTGSSSDVGSYNFVSWYQQHPGKAP...,1287185,AGTGAGGGTTACGCGC-1_contig_1_AGTGAGGGTTACGCGC-1...,QSALTQPASVSGSPGQSITISCTGSSSDVGSYNFVSWYQQHPGKAP...
4,2765,AGTGAGGTCCAACCAA-1_contig_1,"|Deletions: 10, 73||||",AGCTCTGAGAGAGGAGCCTTAGCCCTGGATTCCAAGGCCTATCCAC...,EVQLVESGGGLVKPGGSLRLSCAASGFTFSSYSMNWVRQAPGKGLE...,IGHV3-21*01,IGHD3-10*01,IGHJ4*02,AGTGAGGTCCAACCAA-1_contig_2,|||||,GGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTG...,IGLV2-23*01,IGLJ2*01,QSALTQPASVSGSPGQSITISCTGTSSDVGSYNLVSWYQQHPGKAP...,1287185,AGTGAGGTCCAACCAA-1_contig_1_AGTGAGGTCCAACCAA-1...,QSALTQPASVSGSPGQSITISCTGTSSDVGSYNLVSWYQQHPGKAP...


In [3]:
path = '/content/drive/MyDrive/msc-project-mbalmf01/msc-project-source-code-files-22-23-mbalmf01/notebooks'
os.chdir(path)

import torch
from plm_manipulation import run_ankh, run_ablang, batch_embed, start_ankh

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model, tokenizer = start_ankh(device=device)

prot_col='scfv'
seq_id='seq_id'

#Ankh embedding is GPU RAM intensive.
#Need to split the job up into batches of 5000, which in turn will be split into batches of 100
for i in range(0,df.shape[0], 5000):
  if i+5000 > df.shape[0]:
    tensor_df = batch_embed(df=df.iloc[i:df.shape[0],:], prot_col=prot_col, seq_id=seq_id, batch_size=100, model=model, tokenizer=tokenizer)
    tensor_df.to_csv(f'/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230811_human_paired_seqs_MAPT_ankh{i}.csv', index=None)
    torch.cuda.empty_cache()
    print('completed ankh embedding')
  else:
    tensor_df = batch_embed(df=df.iloc[i:i+5000,:], prot_col=prot_col, seq_id=seq_id, batch_size=100, model=model, tokenizer=tokenizer)
    tensor_df.to_csv(f'/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230811_human_paired_seqs_MAPT_ankh{i}.csv', index=None)
    torch.cuda.empty_cache()
    print(f'completed {i}')

In [4]:
device = 'cpu'
tensor_df = run_ablang(df=df, prot_col='sequence_alignment_aa_heavy', seq_id='seq_id', ablang_model='AbLang_heavy')
tensor_df.to_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230811_human_paired_seqs_MAPT_ablang_heavy.csv', index=None)
tensor_df = run_ablang(df=df, prot_col='sequence_alignment_aa_light', seq_id='seq_id', ablang_model='AbLang_light')
tensor_df.to_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230811_human_paired_seqs_MAPT_ablang_light.csv', index=None)

Using device: cpu


A new version of the following files was downloaded from https://huggingface.co/qilowoq/AbLang_heavy:
- AbLang_roberta_model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Using device: cpu


A new version of the following files was downloaded from https://huggingface.co/qilowoq/AbLang_light:
- AbLang_roberta_model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
from google.colab import runtime

def disconnect_and_delete_runtime():
  """Disconnects and deletes the current runtime."""
  runtime.unassign()

disconnect_and_delete_runtime()